# Grab all the loopy/inlining/odd shapes
Select a couple of examples to see what the pattern is for `stop_sequence` and `shape_meters`.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import dask.dataframe as dd
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
import sys

from segment_speed_utils.project_vars import SEGMENT_GCS, analysis_date

# Temp append so we can import functions from scripts/
sys.path.append("./scripts/")
import prep_stop_segments
import cut_special_stop_segments
import loop_utils

shape_with_utc = "f765b9d12fcca0173b4e3ddbc0374d18"
another_shape = "007cf76062f5957d4e38ea54e624c0ad"

In [2]:
loop_shapes = loop_utils.grab_loop_shapes(analysis_date)

stop_times_with_geom = prep_stop_segments.stop_times_aggregated_to_shape_array_key(
        analysis_date, loop_shapes)

st_loops = stop_times_with_geom.compute()

In [3]:
gdf = (loop_utils.assign_visits_to_stop(st_loops)
       .sort_values(["shape_array_key", "stop_sequence"])
       .reset_index(drop=True)
      )

In [4]:
gdf2 = gdf[gdf.shape_array_key==another_shape]

shape_geometry = gdf2.geometry.iloc[0]
stop_geometry_array = np.array(gdf2.stop_geometry)
stop_sequence_array = np.array(gdf2.stop_sequence)

In [5]:
current_stop_seq = 21

test_segment, test_stops = cut_special_stop_segments.super_project(
    current_stop_seq,
    shape_geometry,
    stop_geometry_array,
    stop_sequence_array,
)

segment_to_plot, stops_to_plot = loop_utils.stop_segment_components_to_geoseries(
    test_segment, test_stops)

loop_utils.plot_segments_and_stops(segment_to_plot, stops_to_plot)

/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point


In [7]:
current_stop_seq = 33

test_segment, test_stops = cut_special_stop_segments.super_project(
    current_stop_seq,
    shape_geometry,
    stop_geometry_array,
    stop_sequence_array,
)

segment_to_plot, stops_to_plot = loop_utils.stop_segment_components_to_geoseries(
    test_segment, test_stops)

loop_utils.plot_segments_and_stops(segment_to_plot, stops_to_plot)

/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point


## Expand to multiple stops within same shape

In [9]:
subset_df = gdf2[gdf2.shape_array_key==another_shape
                ].reset_index(drop=True)

shape_geometry = subset_df.geometry.iloc[0]
stop_geometry_array = np.array(subset_df.stop_geometry)
stop_sequence_array = np.array(subset_df.stop_sequence)

segment_results = []

for stop_seq in stop_sequence_array:
        
    segment_i, _ = cut_special_stop_segments.super_project(
        stop_seq,
        shape_geometry,
        stop_geometry_array,
        stop_sequence_array,
    )
    segment_i_series = loop_utils.stop_segment_components_to_geoseries(
        segment_i, [])
    
    segment_results.append(segment_i_series)


/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
/opt/conda/lib/p

In [10]:
stop_segments = pd.concat(segment_results, 
                          axis=0).reset_index(drop=True)

In [11]:
subset_df = subset_df.assign(
    stop_segments_geometry = stop_segments
)

In [12]:
subset_df[subset_df.stop_segments_geometry.isna()]

,shape_array_key,geometry,feed_key,name,stop_id,stop_sequence,stop_name,stop_geometry,num_visits,stop_segments_geometry


## Expand to multiple shapes

In [ ]:
gdf3 = gdf[gdf.shape_array_key.isin([shape_with_utc, another_shape])]

In [ ]:
def super_cut_stop_segments(
    gdf: gpd.GeoDataFrame, 
    one_shape: str
) -> gpd.GeoDataFrame:
    
    subset_df = (gdf[gdf.shape_array_key==one_shape]
                 .reset_index(drop=True)
                )

    shape_geometry = subset_df.geometry.iloc[0]
    stop_geometry_array = np.array(subset_df.stop_geometry)
    stop_sequence_array = np.array(subset_df.stop_sequence)

    segment_results = []

    for stop_seq in stop_sequence_array:

        segment_i, _ = loopy.super_project(
            stop_seq,
            shape_geometry,
            stop_geometry_array,
            stop_sequence_array,
        )
        segment_i_series = loopy.stop_segment_components_to_geoseries(
            segment_i, [])
    
        segment_results.append(segment_i_series)
    
    stop_segments = (pd.concat(segment_results, axis=0)
                     .reset_index(drop=True)
                    )
    subset_df = subset_df.assign(
        stop_segments_geometry = stop_segments
    )
    
    return subset_df


In [ ]:
results_df = gpd.GeoDataFrame()

results = []

for i in gdf3.shape_array_key.unique():
    segments_for_shape = super_cut_stop_segments(gdf3, i)
    results.append(segments_for_shape)

In [ ]:
results_df = pd.concat(results, axis=0)

In [ ]:
results_df.shape_array_key.value_counts()

In [ ]:
results_df.shape, gdf3.shape

In [ ]:
results_df[results_df.stop_segments_geometry.isna()].shape

In [ ]:
from segment_speed_utils import wrangle_shapes
from shared_utils import rt_utils

In [ ]:
results_df2 = results_df.drop(columns = ["geometry", "stop_geometry"]
                             ).rename(columns = {"stop_segments_geometry": "geometry"})

In [ ]:
segment_geom = gpd.GeoSeries(results_df2.geometry)
CRS = results_df2.crs.to_epsg()
    
geom_parallel = gpd.GeoSeries(
    [i.parallel_offset(30, "right") for i in segment_geom], 
    crs=CRS
)


In [ ]:
geom_arrowized = rt_utils.arrowize_segment(
    geom_parallel, 
    buffer_distance = 20
)

results_df2 = results_df2.assign(
    geometry_arrowized = geom_arrowized
)

In [ ]:
results_df2.plot()

In [ ]:
#results_df2 = wrangle_shapes.add_arrowized_geometry(results_df2)

results_df2.set_geometry("geometry_arrowized").explore(
    "stop_id", tiles="CartoDB Positron")